In [61]:
# Importing necessary libraries for training and visualization
import numpy as np
import pandas as pd
import torch
import torchvision
from time import time
from torchvision import datasets, transforms
from torch import nn, optim
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset
from torch.autograd import Variable
import tensorflow as tf
from sklearn import preprocessing

import matplotlib.pyplot as plt
import tqdm



train_set = pd.read_csv("/Users/cszilagyi2019/Desktop/All Coding/Titanic Predictions/train.csv")
test_set = pd.read_csv("/Users/cszilagyi2019/Desktop/All Coding/Titanic Predictions/test.csv")

In [68]:
#Setting up Training/Validation/Testing through the dataloaders

#Hyperparameters
batch_size = 1
validation_split = 0.2
num_epochs = 20
learning_rate = 0.001

# Create validation split by taking a percentage of the training set:
dataset_size = len(train_set)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))

train_indices, val_indices = indices[split:], indices[:split]

# Use the SubsetRandomSampler to randomly sample the training dataset for 
# training and validation data. We will feed this into the dataloader below.
train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

# Create dataloader objects - will be used during training and inference
# to iterate over the data.
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, \
                                           sampler=train_sampler)
val_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, \
                                         sampler=val_sampler)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, \
                                          shuffle=False)

In [63]:
# Creating the Neural Network Class

class Net(nn.Module):

  def __init__(self, input_size, hidden_size, num_classes):

    super(Net, self).__init__()

    self.fc1 = nn.Linear(input_size, hidden_size)
    self.fc2 = nn.Linear(hidden_size, hidden_size)
    self.output = nn.Linear(hidden_size, num_classes)
    
    # Applying activation function
    self.sigmoid = nn.Sigmoid()
    self.relu = nn.ReLU()

    
  #Forward pass
  def forward(self, x):

    out = self.fc1(x)
    out = self.sigmoid(out)
    out = self.fc2(out)
    out = self.sigmoid(out)
    out = self.output(out)
    #Will use cross entropy
    return out

In [64]:
#Making the dataset

class myDataSet(Dataset):
    
    def __init__(self, file):
        
        #Reading the file into variables
        self.x = file.iloc[:, [2,4,5,6,7,9,11]]
        self.y = file.iloc[:, 1]
        
        #Converting important string values to numbers so they can be fed into the nn
        self.x.replace('male', 0, inplace = True)
        self.x.replace('female', 1, inplace = True)
        self.x.replace('S', 0, inplace = True)
        self.x.replace('C', 1, inplace = True)
        self.x.replace('Q', 2, inplace = True)
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        #Convert to tensor and return
        data = torch.from_numpy(self.x.values.astype(np.float32))
        return data[idx], self.y.iloc[idx]
        

In [65]:
#Setting up Training/Validation/Testing through the dataloaders

#Hyperparameters
batch_size = 10
validation_split = 0.2
num_epochs = 20

# Create validation split by taking a percentage of the training set:
dataset_size = len(train_set)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))

train_indices, val_indices = indices[split:], indices[:split]

# Use the SubsetRandomSampler to randomly sample the training dataset for 
# training and validation data. We will feed this into the dataloader below.
train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)


#Making the datasets
train_dataset = myDataSet(train_set)


# Create dataloader objects - will be used during training and inference
# to iterate over the data.
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, \
                                         sampler=train_sampler)
val_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, \
                                         sampler=val_sampler)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, \
                                          shuffle=False)

In [66]:
#Making the neural network
net = Net(7, 10, 2)

#Cross Entropy Loss/Adam Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), learning_rate=0.001)

In [67]:
#Training loop

for epoch in range(num_epochs):

  for i, (data, survived) in enumerate(train_loader):
    print(i)
    
    # Clear the gradients before performing backpropagation
    optimizer.zero_grad()
    
    # Perform the forward pass - this call of net calls the forward() fn
    outputs = net(data)
    
    # Compute loss on the output of the forward pass and desired label
    loss = criterion(outputs, survived)
    # Compute the gradients with respect to the loss function
    loss.backward()      
    # Update the weights in the neural network using the optimizer/backprop
    optimizer.step()


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61


In [77]:
# Function for getting accuracy, adapted from: 
# https://towardsdatascience.com/a-simple-starter-guide-to-build-a-neural-network-3c2cf07b8d7c

def get_accuracy(loader, my_net):
  correct = 0
  total = 0
    
  for i, (data, survived) in enumerate(loader):
      output = my_net(data)
      predicted = torch.max(outputs.data, 1)  
      total += survived.size(0)               
      correct += (predicted == survived).sum()

  return 100 * correct / total

In [78]:
print(get_accuracy(train_loader, net))

KeyError: 415